In [ ]:
# flask용 모듈
from flask import Flask, request, jsonify
# crawling용 모듈
import requests
import json
import os
import sys
import urllib.request
from urllib.parse import quote
import re
import pandas as pd
import time

application = Flask(__name__)
@application.route("/")
def nationMovie(nation_name):
  nation_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/code/searchCodeList.json?key='+키+'&comCode=2204'
  res = requests.get(nation_list_url)
  text = res.text
  m = json.loads(text)
  for i in range(len(m['codes'])):
    if m['codes'][i]['korNm']==nation_name:
      nation_code=m['codes'][i]['fullCd']
      break


  movie_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key='+키+'&itemPerPage=30&repNationCd='+ str(nation_code) #50개 해봄
  res = requests.get(movie_list_url)
  text = res.text

  d = json.loads(text)

  movie_list = []


  for movie in d['movieListResult']['movieList']:
    movie['repNationNm'] ==nation_name
    movie_list.append([movie['movieCd'],movie['movieNm'], movie['repNationNm']])
    
    
  movie_df = pd.DataFrame(movie_list)
  movie_df.columns = ['movieCd','movieNm','repNationNm']  
  movielist=movie_df['movieNm'].tolist()
    
  #네이버 평점
  client_id = 키
  client_secret = 키

  film_list = []
  for movieNm in movie_df['movieNm']:
    encText = urllib.parse.quote(movieNm)
    url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText + "&display=100" # json 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
      response_body = json.load(response)
      film_list.append(response_body)
      #print(response_body)
    else:
      print("Error Code:" + rescode)
  film_df = pd.DataFrame(film_list)
  film_df.astype({'display': int})

  rating_list = []
#for film in film_df['items']:
  for i in range(film_df.shape[0]):
    for j in range(film_df['display'][i]):
      k=film_df['items'][i][j]['title'].replace("<b>","").replace("</b>","")
      if k in movielist:
        rating_list.append([film_df['items'][i][j]['title'].replace("<b>","").replace("</b>",""),film_df['items'][i][j]['link'],film_df['items'][i][j]['image'],
                            film_df['items'][i][j]['subtitle'].replace("<b>","").replace("</b>",""),
                            film_df['items'][i][j]['pubDate'],film_df['items'][i][j]['actor'],film_df['items'][i][j]['userRating']])
            
  rating_df = pd.DataFrame(rating_list)
  rating_df.columns=['movieNm','link','image','submovieNm','pubDate','actor','userRating']
  rating_df = rating_df.drop_duplicates(['movieNm'], keep='first', ignore_index=True)
  rating_df_sorted = rating_df.sort_values(by=['userRating'], axis=0, ascending=False)
    
  final_recommendation = rating_df_sorted.head(5)
  final_list = final_recommendation.values.tolist()
  return final_list

@application.route("/nation",methods=['POST'])
def nationInput():
    body = request.get_json()  
    print(body)  # SkillPayload 출력
    nation_name = body["action"]["detailParams"]["sys_nation"]["value"]

    answer = nationMovie(nation_name)
    responseBody = {
        "version": "2.0",
        "template": {
            "outputs": [
                {
                    "carousel": {
                        "type": "basicCard",
                        "items": [
                            {
                                "title": answer[0][0],
                                "description": answer[0][3] + ", " + answer[0][4],
                                "thumbnail": {
                                    "imageUrl": answer[0][2]
                                },
                                "buttons": [
                                    {
                                        "action": "webLink",
                                        "label": "보러가기",
                                        "webLinkUrl": answer[0][1]
                                    }
                                ]
                            },
                            {
                                "title": answer[1][0],
                                "description": answer[1][3] + ", " + answer[1][4],
                                "thumbnail": {
                                    "imageUrl": answer[1][2]
                                },
                                "buttons": [
                                    {
                                        "action": "webLink",
                                        "label": "보러가기",
                                        "webLinkUrl": answer[1][1]
                                    }
                                ]
                            },
                            {
                                "title": answer[2][0],
                                "description": answer[2][3] + ", " + answer[2][4],
                                "thumbnail": {
                                    "imageUrl": answer[2][2]
                                },
                                "buttons": [
                                    {
                                        "action": "webLink",
                                        "label": "보러가기",
                                        "webLinkUrl": answer[2][1]
                                    }
                                ]
                            },
                        ]
                    }
                }
            ],
            "quickReplies": [
                {
                    "messageText": "마음에 들어! 추천해줘서 고마워~",
                    "action": "message",
                    "label": "👍"
                 },
                {
                    "messageText": "별로야.. 다시 추천해줘",
                    "action": "message",
                    "label": "👎"
                 }
            ]        
        }
    }

    return jsonify(responseBody)




if __name__ == "__main__":
    application.run(host='0.0.0.0', port=5000)